The model includes both an offensive and defensive universe:
- Offensive: US equities (represented by SPY), international equities (EFA), emerging market equities (EEM) and US aggregate bonds (AGG). More on the seemingly odd inclusion of AGG as an offensive asset in a moment.
- Defensive: US corporate bonds (LQD), US intermediate-term Treasuries (IEF) and US short-term Treasuries (SHY).
- For all assets, at the close on the last trading day of the month, calculate a “momentum score” based on month-end data as follows:
$$12(\frac{p_0}{p_1} – 1) + 4(\frac{p_0}{p_3} – 1) + 2(\frac{p_0}{p_6} – 1) + (\frac{p_0}{p_{12}} – 1)$$
 - Where p0 = the asset price at today’s close, p1 = the asset price at the close of the previous month, etc.
 - Note how this approach overweights more recent months. Doing the math, the most recent 1-month change (p0/p1 – 1) determines 40% of the momentum score, while the most distant month (p11/p12 – 1) determines just ~2%.

- If all four of the offensive assets exhibit positive momentum scores, select the offensive asset with the highest score and allocate 100% of the portfolio to that asset at the close. Note the use of both absolute and relative momentum here, an idea popularized by Gary Antonacci as “Dual Momentum”. Why is that important? Historically, absolute momentum has done well minimizing losses, while relative momentum has helped in generating outsized returns.
- If any of the four offensive assets exhibit negative momentum scores, select the defensive asset (LQD, IEF or SHY) with the highest score (regardless of whether the score is > 0) and allocate 100% of the portfolio to that asset at the close. As we do throughout this site, trades in SHY are assumed to be placed in cash, as it’s more relevant to today’s market given SHY’s low yields coupled with the impact of transaction costs and how frequently this strategy trades.
- Hold the position until the final trading day of the following month.

In [24]:
# -*- coding: utf-8 -*-
"""
Created on Fri May 25 22:22:16 2018

@author: xexx
"""

import pandas as pd
import numpy as np
import pickle

#pull in data from AlphaVantage
from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries(key='NXY0VT9AHBRYGKKC',output_format='pandas')




In [7]:
def historical_data(ticker, outsize = "full"):
    #outsize can be compact and full. 
    #compact returns only the latest 100 data points; full 
    #returns the full-length time series of up to 20 years of historical data. The "compact" option is 
    #recommended if you would like to reduce the data size of each API call
    alphavantage_link = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={0}&apikey=NXY0VT9AHBRYGKKC&datatype=csv&outputsize={1}'.format(ticker, outsize)
    df = pd.read_csv(alphavantage_link)
    return df


In [15]:
VAA = ['AGG', 'EFA', 'EEM']
Maindf = historical_data("SPY")
Maindf.index = pd.to_datetime(Maindf.timestamp) #set the index as the timestamp
#Maindf.index = pd.to_datetime(Maindf.index) #make the index datetime
Maindf = Maindf[['timestamp','close']]
Maindf.columns = ["timestamp","SPY"]


In [16]:
for ticker in VAA:
    temp_df = historical_data(ticker)
    temp_df.index = pd.to_datetime(temp_df.timestamp) #set the index as the timestamp
    temp_df = temp_df[['close']]
    temp_df.columns = [ticker]
    Maindf = Maindf.join(temp_df, how = "left")

Maindf.dropna(inplace = True)
Maindf = Maindf.sort_index()

In [17]:
Maindf = Maindf.sort_index()

In [18]:
Maindf.head(5)

,timestamp,SPY,AGG,EFA,EEM
timestamp,,,,,
2003-09-26,2003-09-26,99.95,102.45,117.00,135.40
2003-09-29,2003-09-29,100.93,102.17,118.50,136.36
2003-09-30,2003-09-30,99.95,102.70,117.00,135.93
2003-10-01,2003-10-01,102.08,102.65,120.84,139.50
2003-10-02,2003-10-02,102.45,102.49,120.65,141.50


In [19]:
Maindf['Year'] = Maindf.index.year
Maindf['Month'] = Maindf.index.month


In [20]:
Maindf['Month Change'] = Maindf['Month'] - Maindf['Month'].shift(1) #shift 1 is to shift down 

In [21]:
Maindf_EOM = Maindf[Maindf['Month Change'] != 0]

In [22]:
Maindf_EOM.head(10)

,timestamp,SPY,AGG,EFA,EEM,Year,Month,Month Change
timestamp,,,,,,,,
2003-09-26,2003-09-26,99.95,102.45,117.00,135.40,2003,9,NaN
2003-10-01,2003-10-01,102.08,102.65,120.84,139.50,2003,10,1.0
2003-11-03,2003-11-03,105.99,101.20,125.23,151.95,2003,11,1.0
2003-12-01,2003-12-01,107.60,101.17,130.28,155.53,2003,12,1.0
2004-01-02,2004-01-02,111.23,101.57,137.15,167.58,2004,1,-11.0
2004-02-02,2004-02-02,113.97,102.35,138.00,167.94,2004,2,1.0
2004-03-01,2004-03-01,116.16,103.29,143.40,178.52,2004,3,1.0
2004-04-01,2004-04-01,113.78,103.74,143.48,176.93,2004,4,1.0
2004-05-03,2004-05-03,112.15,100.36,138.20,159.89,2004,5,1.0


In [25]:
with open('filename.pickle', 'wb') as handle:
    pickle.dump(Maindf, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [26]:
with open('filename.pickle', 'rb') as handle:
    b = pickle.load(handle)